# Rust (1987)
This example is a model of capital replacement. It models a bus fleet maintainer,
who has to decide, at each period, if he wants to keep the engine of each bus, or repair it.
Replacement is expensive, but maintenance can not keep up with wear and tear, and as a result it gets progressively
more expensive to keep in working condition the longer it has driven since last
replacement. The state variable in this model is the odometer reading of a bus. It acts
like a proxy for the accumulated wear and tear on the engine. Planning of the
bus routes is done independently of the maintenance decisions. Each period, a
bus has $p_1$ probability of moving one value up in the state space, $p_2$ of
moving two values, and so on up to $p_k$ of moving $k$ values in the state space.
We place an upper value of $450$ thousand miles on the state space, and consequently
$x=450000$ is absorbing. With a maximum odometer change of three values ($k=3$) and a
discritization of the interval $[0;450000]$ into five bins
we get the following probability matrix for the uncontrolled Markov process
$$
\begin{pmatrix}
 p_1  & p_2 & p_3 & 0  & 0\\
 0  & p_1 & p_2 & p_3  & 0\\
 0  & 0 & p_1 & p_2 & p_3\\
 0 & 0 & 0 & p_1  & p_2+p_3\\
 0 & 0 & 0 & 0  & 1\\
\end{pmatrix}.
$$
In our example, we choose $(p_i)_{i=1}^5$ to be
$$
(0.106915, 0.515449, 0.362065, 0.014345, 0.000858),
$$
and $p_6=1-\Sigma_{i=1}^5 p_i$, and discretize the state space into 175 bins. The discount factor is $0.9999$, and the utilities from keeping the engine or replacing it are determined by
\begin{align}
U(a, x_{t}) =\begin{cases}x_{t}\cdot c  & \text{if } a = \text{keep}\\
RC  & \text{if } a = \text{replace}
\end{cases}.
\end{align}
The state space is very simple to construct, once we find a way to construct the transition matrix.

In [1]:
using StocasEstimators, Plots

In [2]:
# Odometer state
n = 175
o_max = 450
Xval = linspace(0, o_max, n)
p = [0.0937; 0.4475; 0.4459; 0.0127; 0.0002];

F1 = zeros(n, n)
offset = -1
for i = 1:n, j = 1:length(p)
    i+offset+j > n && continue
    F1[i,i+offset+j] = i+offset+j == n ? sum(p[j:end]) : p[j]
end
F = [F1, F1[ones(Int64, n), :]]

S = State("mileage", Xval, F)

Now we construct the LinearUtility type with a parameter vector and a
discount factor close to one.

In [3]:
Z1 = [zeros(n) -0.001*Xval]
Z2 = [-ones(n) zeros(n)]

U = LinearUtility(("repair", "replace"), (Z1, Z2), 0.9999, [11.;2.5])

Discrete state
 * name: mileage
 * n: 175


Stocas.LinearUtility{2,Float64}(("repair", "replace"), ([0.0 0.0; 0.0 -0.00258621; … ; 0.0 -0.447414; 0.0 -0.45], [-1.0 0.0; -1.0 0.0; … ; -1.0 0.0; -1.0 0.0]), 0.9999, [11.0, 2.5], ([0.0, -0.00646552, -0.012931, -0.0193966, -0.0258621, -0.0323276, -0.0387931, -0.0452586, -0.0517241, -0.0581897  …  -1.06681, -1.07328, -1.07974, -1.08621, -1.09267, -1.09914, -1.1056, -1.11207, -1.11853, -1.125], [-11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0  …  -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0, -11.0]), ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), 2)

In [4]:
pgfplots()
plot(S, U, :utility, ylims=(-14, 5))

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 Z M 2.484375 0 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.125 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 L 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 Z M 1.265625 -0.765625 "/>
 
 
<path style="stroke:none;" d="M 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.7

We notice the linearity in the state given a choice. As we will see below, the dynamic nature of the decision problem will make the policy highly nonlinear even if the utility function is linear. This completes our specification.

To solve the model, simply type

In [5]:
V, iters = solve(U, S)

(Stocas.IntegratedValueFunction{Array{Float64,1},Array{Float64,2},Array{Array{Float64,1},1}}([-3651.29, -3651.55, -3651.82, -3652.07, -3652.33, -3652.57, -3652.82, -3653.06, -3653.29, -3653.52  …  -3661.61, -3661.62, -3661.62, -3661.63, -3661.63, -3661.64, -3661.64, -3661.65, -3661.65, -3661.66], [-3651.29, -3651.55, -3651.82, -3652.07, -3652.33, -3652.57, -3652.82, -3653.06, -3653.29, -3653.52  …  -3661.61, -3661.62, -3661.62, -3661.63, -3661.63, -3661.64, -3661.64, -3661.65, -3661.65, -3661.66], [0.0936906 0.447455 … 0.0 0.0; 2.04286e-6 0.0936983 … 0.0 0.0; … ; 0.0496658 0.237198 … 0.0440249 0.425824; 0.0498236 0.237951 … 0.0 0.468165], Array{Float64,1}[[-3651.29, -3651.55, -3651.8, -3652.05, -3652.3, -3652.54, -3652.78, -3653.01, -3653.24, -3653.46  …  -3661.25, -3661.26, -3661.26, -3661.27, -3661.27, -3661.28, -3661.28, -3661.29, -3661.29, -3661.29], [-3651.29, -3651.29, -3651.29, -3651.29, -3651.29, -3651.29, -3651.29, -3651.29, -3651.29, -3651.29  …  -3651.29, -3651.29, -3651.29,

The first output will be a type representing the value function, and the second lets
us know how many iterations were used. We should note that the choice policies are
in $\texttt{U}$ and can be retrieved by using $\texttt{policy(U)}$.
Let us try to solve the model using the different methods.

In [6]:
Vvfi, itersvfi = solve(U, S, Stocas.VFI())
Vnewt, itersnewt = solve(U, S, Stocas.Newton())
Vpoli, iterspoli = solve(U, S, Stocas.Policy())
Vpoly, iterspoly = solve(U, S, Stocas.Poly())

We seem that the last three successfully finds the solution, but when using value function iterations,
we get the warning that the maximum number of iterations was reached without convergence. The default
is 50000 iterations, but this can be changed using the keyword \texttt{iterations}.

In [7]:
V, iter = solve!(U, S, Stocas.VFI(maxiter=250000));
iter

220288

By looking at the output, we see that it converged in 220288 iterations to the same
value function as for the other methods. Timing the four implementations give policy iterations
a slight edge over Newton, but both are very fast compared to VFIs.

Then we simulate data, and construct the $\texttt{Data}$ type.

In [9]:
T, N = 120, 50
D = simulate(U, S, 1, T, N)

Stocas.Data{Int64}([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  120, 120, 120, 120, 120, 120, 120, 120, 120, 120], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Array{Int64,1}[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  5991, 5992, 5993, 5994, 5995, 5996, 5997, 5998, 5999, 6000], [42, 53, 94, 143, 204, 256, 303, 354, 393, 406  …  5684, 5709, 5738, 5755, 5801, 5837, 5851, 5886, 5925, 5989]], [12, 14, 16, 18, 20, 22, 23, 25, 27, 28  …  3, 5, 5, 7, 9, 10, 12, 13, 15, 16], [0; 0; … ; 0; 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  158, 159, 160, 161, 162, 163, 170, 172, 174, 175], [14 0; 94 0; … ; 2 0; 1 3], 6000)

Finally we estimate the model based on the simulated data set

In [12]:
nfxp_results = fit_nfxp(U, S, D)
npl_results = fit_npl(U, S, D)


Results of estimation
 * Method: NPL
 * loglikelihood: -561.0119409694498
 * Estimate: [11.63964686083144,2.793838735501523]
 * Std.err.: [0.8228737720965653,0.25370015424759157]
 Iterations
 * Policy iteration: 

In [16]:
nfxp_results

Results of estimation
 * Method: NFXP
 * loglikelihood: -561.0119409691964
 * Estimate: [11.63965790178408,2.793841747086017]
 * Std.err.: [0.7214905709760383,0.22219748114223198]
 Iterations
 * Maximum likelihood: 6
 * Contractions: 137
 * Newton steps: 28


In [17]:
npl_results

Results of estimation
 * Method: NPL
 * loglikelihood: -561.0119409694498
 * Estimate: [11.63964686083144,2.793838735501523]
 * Std.err.: [0.8228737720965653,0.25370015424759157]
 Iterations
 * Policy iteration: 6
 * Maximum likelihood: 10


In [15]:
plot(S, U, :policy)

Results of estimation
 * Method: NPL
 * loglikelihood: -561.0119409694498
 * Estimate: [11.63964686083144,2.793838735501523]
 * Std.err.: [0.8228737720965653,0.25370015424759157]
 Iterations
 * Policy iteration: 6
 * Maximum likelihood: 10


<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 Z M 2.484375 0 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.125 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 L 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 Z M 1.265625 -0.765625 "/>
 
 
<path style="stroke:none;" d="M 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.7

In [19]:
plotlyjs()
heatmap(collect(1:maximum(D.a)), D.xᵈ, D.nxjᵈ./sum(D.nxjᵈ, 2))